In [1]:
# load all JSONL files in the directory
import os
import json

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/prm_training_data/debug"

first_jsonl = next((f for f in os.listdir(dir_path) if f.endswith(".jsonl")), None)
if first_jsonl:
    with open(os.path.join(dir_path, first_jsonl), "r") as f:
        first_line = f.readline()
        if first_line:
            columns = list(json.loads(first_line).keys())
        else:
            columns = []
else:
    columns = []

print(columns)

['id', 'image_url', 'conversations', 'first_incorrect_step', 'steps_with_score', 'consensus_filtering_algo_label', 'verifier_identified_first_incorrect_step_solution']


In [2]:
from collections import defaultdict, Counter
import json
import os

# First pass: Check schema consistency across all files
print("Checking schema consistency...")
file_schemas = {}
jsonl_files = [f for f in os.listdir(dir_path) if f.endswith(".jsonl")]

for file in jsonl_files:
    with open(os.path.join(dir_path, file), "r") as f:
        first_line = f.readline()
        if first_line:
            schema = set(json.loads(first_line).keys())
            file_schemas[file] = schema

# Check if all schemas are identical
reference_schema = next(iter(file_schemas.values()))
inconsistent_files = []
for file, schema in file_schemas.items():
    if schema != reference_schema:
        inconsistent_files.append((file, schema))

if inconsistent_files:
    print("WARNING: Schema inconsistencies found!")
    for file, schema in inconsistent_files:
        print(f"  {file}: {schema}")
        print(f"    Missing: {reference_schema - schema}")
        print(f"    Extra: {schema - reference_schema}")
else:
    print("✓ All files have consistent schema")

Checking schema consistency...
✓ All files have consistent schema


In [3]:
# Statistics collection
file_stats = {}
all_data = []

print(f"\nProcessing {len(jsonl_files)} files...")

for file in jsonl_files:
    print(f"Processing {file}...")
    
    # Initialize stats for this file
    first_incorrect_step_sections = Counter()
    consensus_filtering_labels = Counter()
    line_count = 0
    
    with open(os.path.join(dir_path, file), "r") as f:
        for line in f:
            item = json.loads(line)
            all_data.append(item)
            line_count += 1
            
            # Collect first_incorrect_step section distribution
            if "first_incorrect_step" in item and item["first_incorrect_step"] is not None:
                if isinstance(item["first_incorrect_step"], (list, tuple)) and len(item["first_incorrect_step"]) >= 1:
                    section_name = item["first_incorrect_step"][0]
                    first_incorrect_step_sections[section_name] += 1
            
            # Collect consensus_filtering_algo_label distribution
            if "consensus_filtering_algo_label" in item and item["consensus_filtering_algo_label"] is not None:
                consensus_filtering_labels[item["consensus_filtering_algo_label"]] += 1
    
    # Store stats for this file
    file_stats[file] = {
        "line_count": line_count,
        "first_incorrect_step_sections": dict(first_incorrect_step_sections),
        "consensus_filtering_labels": dict(consensus_filtering_labels)
    }
    
    print(f"  Lines processed: {line_count}")

# Write flattened data to single file
output_file = os.path.join(dir_path, "final_single_prm_training_data_500k_mc0.8_v1.jsonl")
print(f"\nWriting flattened data to {output_file}...")

with open(output_file, "w") as f:
    for item in all_data:
        f.write(json.dumps(item) + "\n")

print(f"✓ Flattened {len(all_data)} total records to {output_file}")

# Print statistics
print("\n" + "="*60)
print("STATISTICS BY FILE")
print("="*60)

for file, stats in file_stats.items():
    print(f"\n{file}:")
    print(f"  Total records: {stats['line_count']}")
    
    # Get key counts for percentage calculations
    incorrect_count = stats['consensus_filtering_labels'].get('o4-mini_incorrect_and_MC_agrees_and_disagrees', 0)
    correct_count = stats['consensus_filtering_labels'].get('o4-mini_correct_and_MC_agrees', 0)
    unused_count = stats['consensus_filtering_labels'].get('o4-mini_correct_and_MC_disagrees', 0)
    
    reasoning_count = stats['first_incorrect_step_sections'].get('Reasoning', 0)
    visual_count = stats['first_incorrect_step_sections'].get('Visual Elements', 0)
    
    print(f"  First incorrect step sections:")
    for section, count in sorted(stats['first_incorrect_step_sections'].items()):
        if incorrect_count > 0:
            pct = (count / incorrect_count) * 100
            print(f"    {section}: {count} ({pct:.1f}% of incorrect samples)")
        else:
            print(f"    {section}: {count} (no incorrect samples)")
    
    print(f"  Consensus filtering labels:")
    for label, count in sorted(stats['consensus_filtering_labels'].items()):
        print(f"    {label}: {count}")
    
    # Training sample breakdown
    training_total = correct_count + incorrect_count
    print(f"  Training samples breakdown:")
    print(f"    Used for training: {training_total} ({correct_count} correct + {incorrect_count} incorrect)")
    print(f"    Not used for training: {unused_count} (o4-mini_correct_and_MC_disagrees)")
    
    if training_total > 0:
        correct_pct = (correct_count / training_total) * 100
        incorrect_pct = (incorrect_count / training_total) * 100
        print(f"    Training split: {correct_pct:.1f}% correct, {incorrect_pct:.1f}% incorrect")
    else:
        print(f"    Training split: No training samples")

# Overall statistics
print("\n" + "="*60)
print("OVERALL STATISTICS")
print("="*60)

all_sections = Counter()
all_labels = Counter()

for stats in file_stats.values():
    for section, count in stats['first_incorrect_step_sections'].items():
        all_sections[section] += count
    for label, count in stats['consensus_filtering_labels'].items():
        all_labels[label] += count

print(f"Total records across all files: {len(all_data)}")

# Overall key counts for percentage calculations
overall_incorrect_count = all_labels.get('o4-mini_incorrect_and_MC_agrees_and_disagrees', 0)
overall_correct_count = all_labels.get('o4-mini_correct_and_MC_agrees', 0)
overall_unused_count = all_labels.get('o4-mini_correct_and_MC_disagrees', 0)

print(f"Overall first incorrect step sections:")
for section, count in sorted(all_sections.items()):
    if overall_incorrect_count > 0:
        pct = (count / overall_incorrect_count) * 100
        print(f"  {section}: {count} ({pct:.1f}% of incorrect samples)")
    else:
        print(f"  {section}: {count} (no incorrect samples)")

print(f"Overall consensus filtering labels:")
for label, count in sorted(all_labels.items()):
    print(f"  {label}: {count}")

# Overall training sample breakdown
overall_training_total = overall_correct_count + overall_incorrect_count
print(f"Overall training samples breakdown:")
print(f"  Used for training: {overall_training_total} ({overall_correct_count} correct + {overall_incorrect_count} incorrect)")
print(f"  Not used for training: {overall_unused_count} (o4-mini_correct_and_MC_disagrees)")

if overall_training_total > 0:
    overall_correct_pct = (overall_correct_count / overall_training_total) * 100
    overall_incorrect_pct = (overall_incorrect_count / overall_training_total) * 100
    print(f"  Training split: {overall_correct_pct:.1f}% correct, {overall_incorrect_pct:.1f}% incorrect")
else:
    print(f"  Training split: No training samples")


Processing 6 files...
Processing dvqa_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_mc0.8.jsonl...
  Lines processed: 16663
Processing AI2D_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_mc0.8.jsonl...
  Lines processed: 22399
Processing RAVEN_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_mc0.8.jsonl...
  Lines processed: 203115
Processing InfoVQA_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_mc0.8.jsonl...
  Lines processed: 25518
Processing vqav2_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_mc0.8.jsonl...
  Lines processed: 8138
Processing CLEVR_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_mc0.8.jsonl...
  Lines processed: 27326

Writing flattened data to /mnt/fast10/brandon/mmr_rollout_data/prm_training_data/debug/final_single_prm_training_data_500k_mc0.8_v1.jsonl...
✓ Flattened 303159 total records to /mnt/fast10/brandon/mmr_r

In [12]:
import pandas as pd

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/prm_training_data/train/CLEVR_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl"

with open(dir_path, "r") as f:
    lines = f.readlines()
records = [json.loads(line) for line in lines if line.strip()]
df = pd.DataFrame(records)

print(df.head())

                                            messages  \
0  [{'role': 'system', 'content': [{'type': 'text...   
1  [{'role': 'system', 'content': [{'type': 'text...   
2  [{'role': 'system', 'content': [{'type': 'text...   
3  [{'role': 'system', 'content': [{'type': 'text...   
4  [{'role': 'system', 'content': [{'type': 'text...   

                                              images  \
0  [s3://arf-share/arf-ob1-mm-reasoning/training_...   
1  [s3://arf-share/arf-ob1-mm-reasoning/training_...   
2  [s3://arf-share/arf-ob1-mm-reasoning/training_...   
3  [s3://arf-share/arf-ob1-mm-reasoning/training_...   
4  [s3://arf-share/arf-ob1-mm-reasoning/training_...   

                                     id  
0  0004b572-06fb-48a6-bc3e-2363fdfcc600  
1  000bb035-cc5a-44b5-9517-88dcbbce240d  
2  000c4850-5b73-4302-9835-60e07771762d  
3  0011c413-b7e6-4f61-ad8f-0f5942ffc31e  
4  0018cebc-5f77-417e-9185-8abeb076f868  


In [17]:
df.iloc[0]["images"][0]

's3://arf-share/arf-ob1-mm-reasoning/training_data_images/CLEVR-MATH/subset_images/CLEVR_train_067191.png'

In [18]:
import os

cwd_abs_path = os.path.abspath(os.getcwd())
test_path = "s3://arf-share/arf-ob1-mm-reasoning/training_data_images/AI2D/subset_images/706.png"
test_path = test_path.replace("s3://arf-share/arf-ob1-mm-reasoning/", cwd_abs_path + "/")
print(test_path)

/mnt/fast10/brandon/mmr_rollout_data/training_data_images/AI2D/subset_images/706.png


In [1]:
# load all JSONL files in the directory
import os
import json

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/prm_training_data/train"

first_jsonl = next((f for f in os.listdir(dir_path) if f.endswith(".jsonl")), None)
if first_jsonl:
    with open(os.path.join(dir_path, first_jsonl), "r") as f:
        first_line = f.readline()
        if first_line:
            columns = list(json.loads(first_line).keys())
        else:
            columns = []
else:
    columns = []

print(columns)

['messages', 'images', 'id']


In [2]:
from collections import defaultdict, Counter
import json
import os

# First pass: Check schema consistency across all files
print("Checking schema consistency...")
file_schemas = {}
jsonl_files = [f for f in os.listdir(dir_path) if f.endswith(".jsonl")]

for file in jsonl_files:
    with open(os.path.join(dir_path, file), "r") as f:
        first_line = f.readline()
        if first_line:
            schema = set(json.loads(first_line).keys())
            file_schemas[file] = schema

# Check if all schemas are identical
reference_schema = next(iter(file_schemas.values()))
inconsistent_files = []
for file, schema in file_schemas.items():
    if schema != reference_schema:
        inconsistent_files.append((file, schema))

if inconsistent_files:
    print("WARNING: Schema inconsistencies found!")
    for file, schema in inconsistent_files:
        print(f"  {file}: {schema}")
        print(f"    Missing: {reference_schema - schema}")
        print(f"    Extra: {schema - reference_schema}")
else:
    print("✓ All files have consistent schema")

Checking schema consistency...
✓ All files have consistent schema


In [4]:
all_data = []

for file in jsonl_files:
    print(f"Processing {file}...")
    
    line_count = 0
    
    with open(os.path.join(dir_path, file), "r") as f:
        for line in f:
            item = json.loads(line)
            all_data.append(item)
            line_count += 1
    print(f"  Lines processed: {line_count}")

output_file = os.path.join(dir_path, "final_flattened_trl_format_prm_training_data_500k_mc0.8_v1.jsonl")
print(f"\nWriting flattened data to {output_file}...")

with open(output_file, "w") as f:
    for item in all_data:
        f.write(json.dumps(item) + "\n")

print(f"✓ Flattened {len(all_data)} total records to {output_file}")

Processing InfoVQA_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl...
  Lines processed: 19711
Processing vqav2_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl...
  Lines processed: 6818
Processing CLEVR_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl...
  Lines processed: 23487
Processing AI2D_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl...
  Lines processed: 18483
Processing RAVEN_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl...
  Lines processed: 197334
Processing dvqa_final_mc_rollouts_with_all_models_verification_merged_prm_training_data_final_trl_format_mc0.8.jsonl...
  Lines processed: 15501

Writing flattened data to /mnt/fast10/brandon/mmr_rollout_data/prm_training_data/train/final_flattened_trl_format_prm_train

In [7]:
# Push flattened JSONL path to S3 path
!aws s3 cp ./prm_training_data_full_v0 s3://arf-share/arf-ob1-mm-reasoning/prm_training_data_full_v0/ --recursive

upload: prm_training_data_full_v0/final_flattened_trl_format_prm_training_data_500k_mc0.8_v1.jsonl to s3://arf-share/arf-ob1-mm-reasoning/prm_training_data_full_v0/final_flattened_trl_format_prm_training_data_500k_mc0.8_v1.jsonl
